In [ ]:
#######################################
# Imports
#######################################

# Pandas for data wrangling and analysis
import pandas as pd

# Altair for visualization
import altair as alt

# Access to files & folders
import os
from glob import glob

# A function for extracting a DataFrame from mzML (XML with raw data)
from src.mzml_parser import df_from_mzml

In [ ]:
%%javascript // disable scolling in notebook
IPython.OutputArea.auto_scroll_threshold = 9999

In [ ]:
#######################################
# Settings
#######################################
# for resampling of signal data to 
# make it consistent and comparable

# Resample time dimension
scans_per_second = 8

# Savitzky-Golay filter
window_length = 5 # window length (scipy.signal.savgol_filter)
polyorder = 3     # polyorder (scipy.signal.savgol_filter)

In [ ]:
# Define the targets to apply PCI_IS correction
# by reading them in from a csv file
targets = pd.read_csv(
    'targets.csv', sep="\t",
    converters = {'precursor': str, 'product': str}
)

# Add column with unique transition information
targets['transition'] = targets.apply(
    lambda x: f"{x['precursor']}_{x['product']}_{x['polarity']}", axis=1
)

targets

In [ ]:
# Define PCI_IS transition to use and correct with
pciis = {'name': '2F_AEA_02','transition': '350.3_269.2_pos'}

pciis

In [ ]:
# Read in raw signal data from mzml files
mzML_files = glob(os.path.join('mzML','*.mzML'))

# Collect all individual DataFrames as a list
mzML_dfs = [df_from_mzml(mzml_file, scans_per_second, window_length, polyorder) for mzml_file in mzML_files]

# And combine them into a single DataFrame
signal_df = pd.concat(mzML_dfs)

signal_df.head(3)

In [ ]:
# Extract PCI_IS signal
pciis_filter = signal_df['transition'] == pciis['transition']
pciis_data = signal_df[pciis_filter][['file', 'rt','intensity']].copy()

# Change the column name of the intensity to pciis and 
# set the index of the DataFrame to file/rt to be able 
# to stitch the column to the target signal
pciis_data.rename(columns={'intensity': 'pciis'}, inplace=True)
pciis_data = pciis_data.set_index(['file', 'rt'])

In [ ]:
# Combine signal from all targets into a single DataFrame
dfs = []
for tIdx, target in targets.iterrows():
    
    # Extract target signal
    target_filter = signal_df['transition'] == target['transition']
    target_data = signal_df[target_filter].copy()
    
    # Keep index, in case we have multiple targets with the same transition
    target_data['target'] = tIdx
       
    # Prepare index for join with pciis signal using file/rt
    target_data = target_data.set_index(['file', 'rt'])    
    
    # Stitch the PCI_IS signal to the dataframe as a column
    dfs.append(target_data.join(pciis_data['pciis']).reset_index())
    
# Combine the DataFrames again    
df = pd.concat(dfs)

# Extract concentration information from filename
df['concentration'] = df['file'].apply(
    lambda x: x.split("_")[-1].replace(".mzML","")
)

# Extract sample information from filename
df['sample'] = df['file'].apply(
    lambda x: x.split("_")[-2]
)

# Calculate ratio between intensity and pciis
df['ratio'] = df['intensity']/df['pciis']

In [ ]:
from scipy.ndimage import shift

# The phase align code has been used from 
# GitHub: https://github.com/pearsonkyle/Signal-Alignment
# Citation: http://iopscience.iop.org/article/10.3847/1538-3881/aaf1ae/meta
from src.signal_alignment import phase_align

# Define reference (ratio) signal to align to
reference_file = os.path.join('mzML','inj043_PlDiv2_LOW.mzML')

# Extract ratio by target/rt from the reference file
reference__filter = df['file'] == reference_file
reference_ratio = df[reference__filter][['target','rt','ratio']].copy()
reference_ratio = reference_ratio.set_index(['target','rt'])

# Change the column name
reference_ratio.rename(columns={'ratio': 'ratio_reference'}, inplace=True)

# Append a column with the ratio by rt from the reference file
df_aligned = df.set_index(['target','rt']).join(reference_ratio).reset_index().dropna()

# Apply alignement by target/file
aligned_dfs = []
for gIdx, df_grouped in df_aligned.groupby(['target','file']):
                   
    df_grouped['ratio_aligned'] = shift( # apply phase shift
        df_grouped['ratio'], 
        float(phase_align( # calculate phase shift
            df_grouped['ratio_reference'].values, df_grouped['ratio'].values, res=1
        )), 
        mode='constant', 
        cval=0.0
    )
    
    aligned_dfs.append(df_grouped)
    
# Merge them back together
df_aligned = pd.concat(aligned_dfs, ignore_index=True)

# Make all negative values = 0. Negative values could appear
# due to the Savitzky-Golay filter that has been applied
df_aligned['ratio_aligned'] = df_aligned['ratio_aligned'].clip(0)

In [ ]:
# Now we are ready to determine the area of a peak
# We do this with the help of the peak_widths function
# from the scipy package, and the detect_peaks function
# for the package detecta.
from scipy.signal import peak_widths
import warnings

# Duarte, M. (2020) detecta: A Python module to detect events in data. 
# GitHub repository, https://github.com/demotu/detecta.
from detecta import detect_peaks

# Initialize an empty list to collect all peak DataFrames found with
# the required meta-data such as apex, width, start, end, etc.
peak_dfs = []
for tIdx, df_target in df_aligned.groupby('target'):
    
    # Get target details
    t = targets.loc[tIdx]
    
    for fIdx, df_target_file in df_target.groupby('file'):
    
        # By signal type (intensity or ratio)
        for signal_type in ['intensity','ratio','ratio_aligned']:
            peak_index = detect_peaks(
                df_target_file[signal_type],
                mph=0,
                mpd=5,
                threshold=0,
                edge='both',
                kpsh=True
            )

            # Suppress the warnings of the peak_width function
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                peak_width_index = peak_widths(
                    df_target_file[signal_type],
                    peak_index,
                    rel_height=0.98
                )

            # Collect peak meta-data
            peak_rt_apex = df_target_file.iloc[peak_index,]['rt']
            peak_int_apex = df_target_file.iloc[peak_index,][signal_type]

            peak_rt_start = df_target_file.iloc[peak_width_index[2],]['rt']
            peak_int_start = df_target_file.iloc[peak_width_index[2],][signal_type]

            peak_rt_end = df_target_file.iloc[peak_width_index[3],]['rt']
            peak_int_end = df_target_file.iloc[peak_width_index[3],][signal_type]

            peak_width = [ pw[0] - pw[1] for pw in zip(peak_rt_end,peak_rt_start) ]

            peaks_df = pd.DataFrame({
                'peak_rt_apex':peak_rt_apex.tolist(),
                'peak_rt_start':peak_rt_start.tolist(),
                'peak_rt_end':peak_rt_end.tolist(),
                'peak_width':peak_width,
            })

            # Calculate the absolute error between the retention time found of the 
            # apex and the expected retention time as predefined in the targets file.
            peaks_df['abs_rt_error'] = peaks_df['peak_rt_apex'].apply(
                lambda x: abs(x - t.rt)
            )

            # We are only interested in the peak with the smallest retention time error.
            peaks_df = peaks_df.nsmallest(1, 'abs_rt_error')
            
            # Keep track of where the peaks where found, in what 
            # sample, of what target, and based on what signal.
            peaks_df['signal_type'] = signal_type
            peaks_df['target'] = tIdx
            peaks_df['file'] = fIdx            

            # Calculate the peak area
            peaks_df['area'] = peaks_df.apply(
                lambda row:
                    df_target_file[
                        (df_target_file['rt'] >= row['peak_rt_start']) & 
                        (df_target_file['rt'] <= row['peak_rt_end'])
                    ][signal_type].sum()
                ,axis=1
            )
    
            # Keep track of the number of scans used to determine the peak area
            peaks_df['scans'] = peaks_df.apply(
                lambda row:
                    len(df_target_file[
                        (df_target_file['rt'] >= row['peak_rt_start']) & 
                        (df_target_file['rt'] <= row['peak_rt_end'])
                    ][signal_type])
                ,axis=1
            )    
            
            # And add it to the list to combine later
            peak_dfs.append(peaks_df)
               
# Merge them back together
df_peaks = pd.concat(peak_dfs, ignore_index=True)  

# Add concentration
df_peaks['concentration'] = df_peaks['file'].apply(
    lambda x: x.split("_")[-1].replace(".mzML","")
)

# Add sample
df_peaks['sample'] = df_peaks['file'].apply(
    lambda x: x.split("_")[-2]
)

# Save peaks as csv file
df_peaks.to_csv('peaks_found.csv', index=False)

In [ ]:
# Plot settings
plot_height = 180
plot_width = 210
rt_window = 12

# Plot signals of all targets to visualize and compare the impact
# of PCI_IS correction on the signal
for tIdx, target in targets.iterrows():

    # Get target details
    t = targets.loc[tIdx]
    
    # Extract target signal
    df_aligned_window = df_aligned[
        (df_aligned['rt'] >= t.rt - rt_window/2) &
        (df_aligned['rt'] <= t.rt + rt_window/2)
    ].copy()   

    plots = alt.vconcat()
    
    # Show the plots for each concentration ('LOW','MEDIUM','HIGH')
    for concentration in df_aligned_window['concentration'].unique():
        
        # Data frame with peaks of target, grouped by concentration
        df_concentration_peaks = df_peaks[
            (df_peaks['target'] == tIdx) &
            (df_peaks['concentration'] == concentration)
        ]

        # Data frame with signals of target, grouped by concentration
        df_concentration_signal = df_aligned_window[
            (df_aligned_window['target'] == tIdx) &
            (df_aligned_window['concentration'] == concentration)
        ]

        # Original signal
        intensity_plot = alt.Chart(df_concentration_signal).mark_line().encode(
            x='rt', y='intensity', color='sample'
        ).properties(
            width=plot_width, height=plot_height, 
            title=f"intensity ({concentration}) {t['name']}"
        )
        
        # Pciis signal
        intensity_plot = intensity_plot + alt.Chart(df_concentration_signal).mark_line(
            strokeWidth=0.5
        ).encode(
            x='rt', y='pciis', color='sample'
        ).properties(
            width=plot_width, height=plot_height
        )        

        # Add rt of target (red verticle line)
        intensity_plot = intensity_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'intensity']).mark_rule(
                color='red', strokeWidth=2
            ).encode(
                alt.X('mean(peak_rt_apex)',
                  title='rt')        
            )

        # Add area window (start) of target (red verticle line)
        intensity_plot = intensity_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'intensity']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_start)'
            )

        # Add area window (end) of target (red verticle line)
        intensity_plot = intensity_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'intensity']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_end)'
            )   

        # Ratio
        unaligned_plot = alt.Chart(df_concentration_signal).mark_line().encode(
            x='rt', y='ratio', color='sample'
        ).properties(
            width=plot_width, height=plot_height, 
            title=f"ratio ({concentration}) {t['name']}"
        )

        # Add rt of target (red verticle line)
        unaligned_plot = unaligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio']).mark_rule(
                color='red', strokeWidth=2
            ).encode(
                alt.X('mean(peak_rt_apex)',
                  title='rt')        
            )

        # Add area window (start) of target (red verticle line)
        unaligned_plot = unaligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_start)'
            )

        # Add area window (end) of target (red verticle line)
        unaligned_plot = unaligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_end)'
            )    

        # Ratio aligned + target rt
        aligned_plot = alt.Chart(df_concentration_signal).mark_line().encode(
            x='rt', y='ratio_aligned', color='sample'
        ).properties(
            width=plot_width, height=plot_height, 
            title=f"ratio aligned ({concentration}) {t['name']}"
        )

        # Add rt of target (red verticle line)
        aligned_plot = aligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio_aligned']).mark_rule(
                color='red', strokeWidth=2
            ).encode(
                alt.X('mean(peak_rt_apex)',
                  title='rt')        
            )

        # Add area window (start) of target (red verticle line)
        aligned_plot = aligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio_aligned']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_start)'
            )

        # Add area window (end) of target (red verticle line)
        aligned_plot = aligned_plot + alt.Chart(
            df_concentration_peaks[df_concentration_peaks['signal_type'] == 'ratio_aligned']).mark_rule(
                color='grey', strokeWidth=2
            ).encode(
                x='min(peak_rt_end)'
            )    

        plots = alt.vconcat(plots, (intensity_plot | unaligned_plot | aligned_plot))
                
    # Display the plots        
    plots.display()    

    # Uncomment to save to disk
    # plots.save(f"plots/{t['name']}.png", scale_factor=2)

In [ ]:
# Plot variation in area based on intensity and PCI_IS corrected ratio
for tIdx, target in targets.iterrows():
    
    # Get target details
    t = targets.loc[tIdx] 

    plots = []
    for signal_type in ['intensity','ratio_aligned']:

      # Filtered data source
      peaks_filter = f"target == {tIdx} and signal_type == '{signal_type}'"    
      source = df_peaks.query(peaks_filter)

      error_points = alt.Chart(source).mark_point(filled=True).encode(
        x=alt.X('area:Q', aggregate='mean', scale=alt.Scale(domain=[source['area'].min(),source['area'].max()])),
        y=alt.Y('concentration:N', sort=alt.SortField('sample')),
        color=alt.Color("concentration", legend=None)
      ).properties(width=350, height=70, title=f"{t['name']}: mean area + error bars ({signal_type})")    

      # Mean area by concentration error bars
      error_bars = alt.Chart(source).mark_errorbar(extent='ci').encode(
        x=alt.X('area:Q', axis=alt.Axis(labels=False), scale=alt.Scale(domain=[source['area'].min(),source['area'].max()])),
        y=alt.Y('concentration:N', sort=alt.SortField('sample')),
        color=alt.Color("concentration", legend=None)
      ).properties(width=350, height=70)

      # (error_points + error_bars).display()
      plots.append((error_points + error_bars))

    (plots[0] | plots[1]).display()
    